# Datos Electorales. ARGENTINA.


In [1]:
# Optimal pandas import
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

pd.options.display.max_colwidth = -1
pd.options.display.max_rows = 999

## Datos de distritos

In [3]:
from unidecode import unidecode

COD_PROV = pd.read_csv('./../data/codigos/COD_PROVINCIAS.csv', ';', encoding = 'latin-1').rename(columns = {'NOMBRE': 'PROVINCIA'})
COD_DEP = pd.read_csv('./../data/codigos/COD_DEPARTAMENTOS.csv', ';', encoding = 'latin-1').rename(columns = {'NOMBRE': 'DEPARTAMENTO'})

COD_PROV['PROV_REF_ID'] = COD_PROV['CODIGO PROVINCIA'].astype(int)#.astype(str)
COD_DEP['PROV_REF_ID'] = COD_DEP['CODIGO PROVINCIA'].astype(int)

#insertar 0 en nombres de comuna, ej. Comuna 05
COD_DEP['dpto_merger_str'] = COD_DEP['DEPARTAMENTO'].str.replace('12 ', 'Doce'
                                            ).str.replace('1° ', 'Primero'
                                            ).str.replace('2 ', 'Dos '
                                            ).str.replace('25', 'Veinticinco'
                                            ).str.replace('9 ', 'Nueve ')

COD_DEP['dpto_merger_str'] = [unidecode(latin_str).lower() for latin_str in COD_DEP['DEPARTAMENTO']]
COD_DEP.loc[:8, 'dpto_merger_str'] = [s[:7] + '0' + s[7:] for s in COD_DEP.loc[:8]['dpto_merger_str'].values]

# Fix differences for BUENOS AIRES dptos
COD_DEP['dpto_merger_str'] = COD_DEP['dpto_merger_str'].str.replace('matanza', 'la matanza'
                                            ).str.replace('j. c. paz', 'jose c. paz'
                                            ).str.replace('roque saenz pena', 'presidente roque saenz pena '
                                            ).str.replace('coronel rosales', 'coronel de marina l. rosales'
                                            ).str.replace('general lamadrid', 'general la madrid'
                                            ).str.replace('las heras', 'general las heras'
                                            ).str.replace('general madariaga', 'general juan madariaga'
                                            ).str.replace('gonzales chaves', 'adolfo gonzales chaves'
                                            ).str.replace('f. ameghino', 'florentino ameghino'
                                            ).str.replace('juarez', 'benito juarez'
                                            ).str.replace('arrecifes', 'bartolome mitre'
                                            ).str.replace('lezama', 'chascomus')

geo_ref = pd.read_csv('./../data/censo_geo/radio_ref.csv')
geo_ref['dpto_merger_str'] = geo_ref['NOMDPTO'].str.replace('12 ', 'Doce'
                                            ).str.replace('1° ', 'Primero'
                                            ).str.replace('2 ', 'Dos '
                                            ).str.replace('25', 'Veinticinco'
                                            ).str.replace('9 ', 'Nueve ')

geo_ref['dpto_merger_str'] = geo_ref['dpto_merger_str'].str.strip()
geo_ref['dpto_merger_str'] = [unidecode(latin_str).lower() for latin_str in geo_ref['dpto_merger_str']]
# geo_ref['dpto_merger_str'] = geo_ref['dpto_merger_str'].str.replace('arrecifes', 'bartolome mitre')

geo_ref.head()

# Las cantidades matchean. Bien!
# Tambien los codigos de provincia.
# geo_ref.groupby('PROV_REF_ID')['DPTO'].nunique()
# COD_DEP.groupby('IDPROV')['DEPARTAMENTO'].nunique()

merged = COD_DEP.merge(geo_ref[['PROV_REF_ID','PROV','DPTO', 'dpto_merger_str']].drop_duplicates(), how = 'left')
merged = merged.dropna() # perdemos dptos no matcheados del interior

IDcols = ['PROV_REF_ID', 'PROV', 'DPTO']
merged[IDcols] = merged[IDcols].applymap(int).applymap(str)


# Las provincias completas son CABA y GBA. Resto del pais hay como 30/350 dptos que no matchean
dpto_ref_BA = merged.loc[merged.PROV_REF_ID.isin(['1', '2'])].set_index('DEPARTAMENTO')[['DPTO', 'dpto_merger_str']]
display(dpto_ref_BA.sample(5))
# Ya esta todo linkeado con DPTOS geo, se pueden hacer mapas.

,DPTO,dpto_merger_str
DEPARTAMENTO,,
CAñUELAS,6134,canuelas
DOLORES,6238,dolores
JUNIN,6413,junin
CAPITAN SARMIENTO,6140,capitan sarmiento
ARRECIFES,6077,bartolome mitre


# Data Electoral

# 2015

In [ ]:
# Nombres de listas
COD_PARTIDOS = pd.read_csv('./../data/codigos/COD_PARTIDOS_2015.csv', ';', encoding = 'latin-1')[[u'Codigo Partido', u'Denominacion']]
COD_PARTIDOS.columns = [unidecode(col) for col in COD_PARTIDOS.columns] # atender  Esta tilde
COD_PARTIDOS['Denominacion'] = COD_PARTIDOS['Denominacion'].str.strip()
COD_PARTIDOS['Denominacion'] = COD_PARTIDOS['Denominacion'].str.replace('ALIANZA CAMBIEMOS', 'CAMBIEMOS')
COD_PARTIDOS['Denominacion'] = COD_PARTIDOS['Denominacion'].str.replace('CAMBIEMOS BUENOS AIRES', 'CAMBIEMOS')

## Dip Nacs. 2015

In [4]:
#Load data

DN_2015 = pd.concat([pd.read_csv('./../data/escrutinio/2015/FMESDN_0101.csv', ';'),
                            pd.read_csv('./../data/escrutinio/2015/FMESDN_0202.csv', ';'),
                            pd.read_csv('./../data/escrutinio/2015/FMESDN_0313.csv', ';'),
                            pd.read_csv('./../data/escrutinio/2015/FMESDN_1424.csv', ';')])
DN_2015['ELECCION'] = 'PV_2015'
# DN_2015.sum()

 # mergear
DN_2015 = DN_2015.merge(COD_PARTIDOS, left_on = u'CODIGO VOTOS', right_on = u'Codigo Partido')

# mergear parte geografica
DN_2015 = DN_2015.merge(COD_PROV).merge(COD_DEP, on = ['CODIGO DEPARTAMENTO', u'PROV_REF_ID'])
# #fix
DN_2015.columns = DN_2015.columns.str.strip()
DN_2015['circuito'] = DN_2015['CODIGO CIRCUITO'].astype(str).str.strip().str.zfill(4)


## Gobernador 2015

In [20]:
#Load data
GO_2015 = pd.read_csv('./../data/escrutinio/2015/FMESGO_0202.csv', ';')
GO_2015['ELECCION'] = 'PV_2015'
# DN_2015.sum()
 # mergear
GO_2015 = GO_2015.merge(COD_PARTIDOS, left_on = u'CODIGO VOTOS', right_on = u'Codigo Partido')
# mergear parte geografica
GO_2015 = GO_2015.merge(COD_PROV).merge(COD_DEP, on = ['CODIGO DEPARTAMENTO', u'PROV_REF_ID'])
# #fix
GO_2015.columns = GO_2015.columns.str.strip()
GO_2015['circuito'] = GO_2015['CODIGO CIRCUITO'].astype(str).str.strip().str.zfill(4)


## Municipales 2015

In [22]:
#Load data
MU_2015 = pd.read_csv('./../data/escrutinio/2015/FMESMU_0202.csv', ';')
MU_2015['ELECCION'] = 'PV_2015'
# DN_2015.sum()
 # mergear
MU_2015 = MU_2015.merge(COD_PARTIDOS, left_on = u'CODIGO VOTOS', right_on = u'Codigo Partido')
# mergear parte geografica
MU_2015 = MU_2015.merge(COD_PROV).merge(COD_DEP, on = ['CODIGO DEPARTAMENTO', u'PROV_REF_ID'])
# #fix
MU_2015.columns = MU_2015.columns.str.strip()
MU_2015['circuito'] = MU_2015['CODIGO CIRCUITO'].astype(str).str.strip().str.zfill(4)


### Cargar Paso 2017

In [5]:
paso_2017 = pd.concat([pd.read_csv('./../data/escrutinio/2017/paso2017-FMESDN_0101.csv', ';'),
                            pd.read_csv('./../data/escrutinio/2017/paso2017-FMESDN_0202.csv', ';'),
                            pd.read_csv('./../data/escrutinio/2017/paso2017-FMESDN_0313.csv', ';'),
                            pd.read_csv('./../data/escrutinio/2017/paso2017-FMESDN_1424.csv', ';')])
paso_2017['ELECCION'] = 'paso_2017'

COD_PARTIDOS = pd.read_csv('./../data/codigos/COD_PARTIDOS_2017_paso.csv', ';', encoding = 'latin-1')[[u'Código Partido', u'Denominacion']]
COD_PARTIDOS.columns = [unidecode(col) for col in COD_PARTIDOS.columns]
COD_PARTIDOS['Denominacion'] = COD_PARTIDOS['Denominacion'].str.strip()
COD_PARTIDOS['Denominacion'] = COD_PARTIDOS['Denominacion'].str.replace('CAMBIEMOS BUENOS AIRES', 'CAMBIEMOS')
COD_PARTIDOS['Denominacion'] = COD_PARTIDOS['Denominacion'].str.replace('VAMOS JUNTOS', 'CAMBIEMOS')
COD_PARTIDOS['Denominacion'] = COD_PARTIDOS['Denominacion'].str.replace(u'UNIDAD PORTEÑA', 'UNIDAD CIUDADANA')

paso_2017 = paso_2017.merge(COD_PARTIDOS, left_on = u'CODIGO VOTOS', right_on = u'Codigo Partido')
paso_2017 = paso_2017.merge(COD_PROV).merge(COD_DEP, on = ['CODIGO DEPARTAMENTO', u'PROV_REF_ID'])
# #fix
paso_2017.columns = paso_2017.columns.str.strip()
paso_2017['circuito'] = paso_2017['CODIGO CIRCUITO'].astype(str).str.strip().str.zfill(4)

### Cargar DN 2017

In [6]:

DN_2017 = pd.concat([pd.read_csv('./../data/escrutinio/2017/FMESDN_0101.csv', ';'),
                            pd.read_csv('./../data/escrutinio/2017/FMESDN_0202.csv', ';'),
                            pd.read_csv('./../data/escrutinio/2017/FMESDN_0313.csv', ';'),
                            pd.read_csv('./../data/escrutinio/2017/FMESDN_1424.csv', ';')])
DN_2017['ELECCION'] = 'DN_2017'

COD_PARTIDOS = pd.read_csv('./../data/codigos/COD_PARTIDOS_2017_paso.csv', ';', encoding = 'latin-1')[[u'Código Partido', u'Denominacion']]
COD_PARTIDOS.columns = [unidecode(col) for col in COD_PARTIDOS.columns]
COD_PARTIDOS['Denominacion'] = COD_PARTIDOS['Denominacion'].str.strip()
COD_PARTIDOS['Denominacion'] = COD_PARTIDOS['Denominacion'].str.replace('CAMBIEMOS BUENOS AIRES', 'CAMBIEMOS')
COD_PARTIDOS['Denominacion'] = COD_PARTIDOS['Denominacion'].str.replace('VAMOS JUNTOS', 'CAMBIEMOS')
COD_PARTIDOS['Denominacion'] = COD_PARTIDOS['Denominacion'].str.replace(u'UNIDAD PORTEÑA', 'UNIDAD CIUDADANA')

DN_2017 = DN_2017.merge(COD_PARTIDOS, left_on = u'CODIGO VOTOS', right_on = u'Codigo Partido')
DN_2017 = DN_2017.merge(COD_PROV).merge(COD_DEP, on = ['CODIGO DEPARTAMENTO', u'PROV_REF_ID'])
# #fix
DN_2017.columns = DN_2017.columns.str.strip()
DN_2017['circuito'] = DN_2017['CODIGO CIRCUITO'].astype(str).str.strip().str.zfill(4)

In [7]:
## Un poquito de numerologia, porque dan muy redondo algunos numeros en la PASO 17 Buenos Aires.

# Cambiemos tiene 20.00% y UC 10.009986% de los votantes.

# (3310366 * 2) - 6555803
# 3310366*0.000001
# 6555803/64929
# 3310366/0.10099 #32776326
# 32776326 - 32775900.99009901
# 3310366/32776326 #0.100999
# 0.10099869033521329 - 0.101
# -1.30966478671668e-06 * 32776326
# 6555803/32776326 #0.200016
# 0.00001640818437064*32776326

## Armar cuadro resumen nacional

In [8]:
# data_eleccion = DN_2017

# total_de_electores = 33502945 # Oct 17
# # total_de_electores = 32776326 # Aug 17
# # total_de_electores = 31754774 # Oct 15

# cuadro_nac = data_eleccion.groupby('Denominacion').sum().sort_values(by = 'VOTOS', ascending = False)#/total_de_electores 
# # SORPRENDENTE. Con paso 17
# # Como puede ser que sean tan redondos los numeros?
# # Estamos hablando de 6555803
# cuadro_nac.head(10)

# Cuadro resumen nacional. Elecciones 2015, p2017 y 2017.

In [9]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [13]:
# by_col = ['dpto_merger_str', 'circuito']
by_col = ['dpto_merger_str']
results_summary = []

for i, data_eleccion in enumerate([DN_2015, paso_2017, DN_2017]):

    main = [['ELECTORES', 'VOTANTES', 'ALIANZA FRENTE PARA LA VICTORIA', 'CAMBIEMOS',
         'ALIANZA UNIDOS POR UNA NUEVA ALTERNATIVA (UNA)', 'VOTOS BLANCOS',
         'ALIANZA FRENTE DE IZQUIERDA Y DE LOS TRABAJADORES'],
        ['ELECTORES', 'VOTANTES', 'CAMBIEMOS', 'UNIDAD CIUDADANA', '1PAIS',
         'FRENTE JUSTICIALISTA', 'VOTOS BLANCOS',
         'FRENTE DE IZQUIERDA Y DE LOS TRABAJADORES'],
        ['ELECTORES', 'VOTANTES', 'CAMBIEMOS', 'UNIDAD CIUDADANA',
         'FRENTE JUSTICIALISTA', 'FRENTE DE IZQUIERDA Y DE LOS TRABAJADORES',
         '1PAIS', 'VOTOS BLANCOS']][i]
    name = ['DN_2015', 'paso_2017', 'DN_2017'][i]
    ppales_listas = data_eleccion.loc[data_eleccion.Denominacion.isin(main)]
    total_de_electores = data_eleccion.groupby('Denominacion').sum().loc['ELECTORES']['VOTOS']

    # Seleccionar provincias
    ppales_listas_prov = ppales_listas.loc[ppales_listas['PROVINCIA'].isin(['BUENOS AIRES', 'CIUDAD AUTONOMA DE BS.AS.'])]
#     ppales_listas_prov = ppales_listas.loc[ppales_listas['PROVINCIA'].isin(['NEUQUEN'])]

    cuadro = ppales_listas_prov.groupby(['Denominacion','PROV_REF_ID']+ by_col).sum()[['VOTOS']].sort_values(by = 'VOTOS', ascending = False)
    cuadro = cuadro.unstack([0, 1])

    # Crear tabla porcentual
    cuadro_pct = cuadro.copy()
    cuadro_pct['VOTOS'] = cuadro['VOTOS']/cuadro['VOTOS']['VOTANTES']
    cuadro_pct.columns = cuadro_pct.columns.set_levels(['VOTOS_pct'], level = 0)
    cuadro_pct = pd.concat([cuadro_pct], keys=[name], names=['Eleccion'], axis = 1)

    # Asignar sufijo cnt para cuenta de votos (no %, pct)
    cuadro.columns = cuadro.columns.set_levels(['VOTOS_cnt'], level = 0)

        
    cuadro = pd.concat([cuadro], keys=[name], names=['Eleccion'], axis = 1)

    results_summary += [pd.concat([cuadro, cuadro_pct], axis = 1)]


results_summary = pd.concat(results_summary, axis = 1).stack()

# if len(by_col) == 2:
#     results_summary = results_summary.stack(-2, -1)
# elif len(by_col) == 1:
#     results_summary = results_summary.stack()

In [14]:
# Ejemplo
import seaborn as sns
cm = sns.light_palette("green", as_cmap=True)

scale = 'cnt'
s = results_summary#['DN_2017']['VOTOS_'+scale]
ordenh = s.sum().sort_values(ascending = False).index
ordenv = s.sum(1).sort_values(ascending = False).index
# (100*s)

#order
s = s.reindex(ordenv).reindex(ordenh,axis = 1)
# Append totals
# s = s.append(s.sum(), ignore_index=True)
s.style.background_gradient(cmap=cm, axis = 0)


In [15]:
# # Ejemplo
# s.to_csv('./tabla_resumen.csv')
# s.head()

In [16]:
## Guardar
# ['dpto_merger_str'][-1][:4]

In [17]:
results_summary.to_csv('./../data/tablas_votos/VOTOS_2015_17_'+by_col[-1][:4]+'.csv')

In [18]:
# como cargarlo
df = pd.read_csv('./../data/tablas_votos/VOTOS_2015_17_'+by_col[-1][:4]+'.csv', header = [0, 1, 2], index_col=list(range(len(by_col) + 1)))
df.head()

Eleccion                                                                     DN_2015  \
                                                                           VOTOS_cnt   
Denominacion                       ALIANZA FRENTE DE IZQUIERDA Y DE LOS TRABAJADORES   
dpto_merger_str        PROV_REF_ID                                                     
adolfo alsina          2            128.0                                              
adolfo gonzales chaves 2            138.0                                              
alberti                2            99.0                                               
almirante brown        2            16612.0                                            
avellaneda             2            11284.0                                            

Eleccion                                                            \
                                                                     
Denominacion                       ALIANZA FRENTE PARA LA VICTORIA   
dpto_merger_str        PROV_REF_ID                                   
adolfo alsina          2            2688.0                           
adolfo gonzales chaves 2            3347.0                           
alberti                2            2701.0                           
almirante brown        2            131299.0                         
avellaneda             2            81333.0                          

Eleccion                                                                           \
                                                                                    
Denominacion                       ALIANZA UNIDOS POR UNA NUEVA ALTERNATIVA (UNA)   
dpto_merger_str        PROV_REF_ID                                                  
adolfo alsina          2            1156.0                                          
adolfo gonzales chaves 2            1035.0                                          
alberti                2            732.0                                           
almirante brown        2            64659.0                                         
avellaneda             2            38981.0                                         

Eleccion                                                          \
                                                                   
Denominacion                       CAMBIEMOS ELECTORES  VOTANTES   
dpto_merger_str        PROV_REF_ID                                 
adolfo alsina          2            4957.0    13189.0   10543.0    
adolfo gonzales chaves 2            2279.0    10420.0   8846.0     
alberti                2            3009.0    8859.0    7308.0     
almirante brown        2            82618.0   419609.0  320424.0   
avellaneda             2            70034.0   289664.0  219028.0   

Eleccion                                          \
                                                   
Denominacion                       VOTOS BLANCOS   
dpto_merger_str        PROV_REF_ID                 
adolfo alsina          2            1291.0         
adolfo gonzales chaves 2            1694.0         
alberti                2            916.0          
almirante brown        2            24019.0        
avellaneda             2            11228.0        

Eleccion                                                                              \
                                                                           VOTOS_pct   
Denominacion                       ALIANZA FRENTE DE IZQUIERDA Y DE LOS TRABAJADORES   
dpto_merger_str        PROV_REF_ID                                                     
adolfo alsina          2            0.012141                                           
adolfo gonzales chaves 2            0.015600                                           
alberti                2            0.013547                                           
almirante brown        2            0.051844                                           
avellaneda             2